### Stock ID

In [47]:
STOCK_ID = [
    "50200",
    "80580",
    "69200",
    "80010",
    "69520",
    "43930",
    "91040",
    "79740",
    "72030",
    "24320",
    "46890",
    "40630",
    "51080",
    "96970",
    "99830",
    "99840",
    "45680",
    "18120",
    "70110",
    "88010",
    "36350",
    "48160",
    "83060",
    "54110",
    "68570"
]

def stock_id_preprocessing(stock_ids):
    result = []
    for stock in stock_ids:
        if stock.endswith("0"):
            stock = stock[:-1] + ".T"
        result.append(stock)
    return result

STOCK_ID = stock_id_preprocessing(STOCK_ID)
print(STOCK_ID)

['5020.T', '8058.T', '6920.T', '8001.T', '6952.T', '4393.T', '9104.T', '7974.T', '7203.T', '2432.T', '4689.T', '4063.T', '5108.T', '9697.T', '9983.T', '9984.T', '4568.T', '1812.T', '7011.T', '8801.T', '3635.T', '4816.T', '8306.T', '5411.T', '6857.T']


# ROAの底打ち傾向 - ROA Bottoming Trend

In [57]:
from itertools import product
import yfinance as yf
import pandas as pd
import numpy as np

class ROABottomingTrendFuzzy:
    def __init__(self, MAP_ROA, MAP_TREND, OUTPUT_LEVELS, RULE_TABLE):
        self.MAP_ROA = MAP_ROA
        self.MAP_TREND = MAP_TREND
        self.OUTPUT_LEVELS = OUTPUT_LEVELS
        self.RULE_TABLE = RULE_TABLE

    def linear_fuzzy(self, x, x1, y1, x2, y2):
        if x <= min(x1, x2):
            return y1 if x1 < x2 else y2
        if x >= max(x1, x2):
            return y2 if x1 < x2 else y1
        return y1 + (y2 - y1) * (x - x1) / (x2 - x1)

    def fetch_roa_multi_year(self, stock_id):
        ticker = yf.Ticker(stock_id)
        financials = ticker.financials
        balance_sheet = ticker.balance_sheet

        if financials is None or balance_sheet is None:
            return None
        if financials.empty or balance_sheet.empty:
            return None

        roa_by_year = {}

        for col in financials.columns:
            try:
                # ---- safe get ----
                if "Net Income" not in financials.index:
                    continue
                if "Total Assets" not in balance_sheet.index:
                    continue

                net_income = financials.loc["Net Income", col]
                total_assets = balance_sheet.loc["Total Assets", col]

                # ---- NaN / None check ----
                if pd.isna(net_income) or pd.isna(total_assets):
                    continue

                # ---- zero / invalid check ----
                if total_assets == 0:
                    continue

                roa = net_income / total_assets

                # ---- final NaN / inf guard ----
                if np.isnan(roa) or np.isinf(roa):
                    continue

                roa_by_year[col.year] = roa

            except Exception:
                continue

        if not roa_by_year:
            return None

        # sort by year
        return [roa_by_year[y] for y in sorted(roa_by_year)]

    def in_range(self, x, a, b):
        return a < x < b

    def fuzzy_product(self, roa_value, slope):
        '''
        Fuzzification of ROA and Trend (slope)
        Inputs:
            roa_value: float, ROA value
            slope: float, slope of ROA trend
        Outputs:
            ROA_FUZZY: dict, fuzzy membership of ROA
            TREND_FUZZY: dict, fuzzy membership of Trend
        '''
        
        roa_raw = []
        trend_raw = []
        
        # ---------- ROA fuzzification ----------
        for key, (a, b) in self.MAP_ROA.items():
            if self.in_range(roa_value, a, b):
                roa_raw.append((key, a, b))
        
        ROA_FUZZY = {}

        if len(roa_raw) == 1:
            ROA_FUZZY[roa_raw[0][0]] = 1.0

        elif len(roa_raw) == 2:
            (k1, a1, b1), (k2, a2, b2) = roa_raw
            left = max(a1, a2)
            right = min(b1, b2)

            ROA_FUZZY[k1] = self.linear_fuzzy(roa_value, left, 1, right, 0)
            ROA_FUZZY[k2] = self.linear_fuzzy(roa_value, left, 0, right, 1)

        else:
            ROA_FUZZY["UNKNOWN"] = 0.0

        # ---------- TREND fuzzification ----------
        for key, (a, b) in self.MAP_TREND.items():
            if self.in_range(slope, a, b):
                trend_raw.append((key, a, b))
        
        TREND_FUZZY = {}

        if len(trend_raw) == 1:
            TREND_FUZZY[trend_raw[0][0]] = 1.0

        elif len(trend_raw) == 2:
            (k1, a1, b1), (k2, a2, b2) = trend_raw
            left = max(a1, a2)
            right = min(b1, b2)

            TREND_FUZZY[k1] = self.linear_fuzzy(slope, left, 1, right, 0)
            TREND_FUZZY[k2] = self.linear_fuzzy(slope, left, 0, right, 1)

        else:
            TREND_FUZZY["UNKNOWN"] = 0.0

        return ROA_FUZZY, TREND_FUZZY
    
    def Rule(self, roa, trend):
        return self.RULE_TABLE.get((roa, trend), ("UNKNOWN", 0))
    
    def infer_rules(self, roa_fuzzy, trend_fuzzy):
        combine_rules = []

        for (roa_label, roa_w), (trend_label, trend_w) in product(
            roa_fuzzy.items(), trend_fuzzy.items()
        ):
            rule_name, rule_score = self.Rule(roa_label, trend_label)

            firing_strength = min(roa_w, trend_w)  # AND = min

            combine_rules.append({
                "roa": roa_label,
                "trend": trend_label,
                "label": rule_name,
                "score": rule_score,
                "weight": firing_strength
            })

        return combine_rules
    
    def defuzzify_sugeno(self, combine_rules):
        numerator = 0.0
        denominator = 0.0

        for r in combine_rules:
            numerator += r["weight"] * r["score"]
            denominator += r["weight"]

        if denominator == 0:
            return 0

        return numerator / denominator

    def slope_of_list(self, data):
        n = len(data)
        if n < 2:
            return 0.0

        x = np.arange(n)
        y = np.array(data)

        x_mean = np.mean(x)
        y_mean = np.mean(y)

        numerator = np.sum((x - x_mean) * (y - y_mean))
        denominator = np.sum((x - x_mean) ** 2)

        if denominator == 0:
            return 0.0

        slope = numerator / denominator
        return slope

    def map_fuzzy_output_centroid(self, score):
        label = min(
            OUTPUT_LEVELS.items(),
            key=lambda x: abs(score - x[1])
        )
        return label

    def ROA_Bottoming_Trend(self, stock_id, get_level_label=False):
        data = self.fetch_roa_multi_year(stock_id)
        min_value = min(data)
        min_index = data.index(min_value)
        if min_index == len(data) - 1:
            # Left
            slope = self.slope_of_list(data)
        if min_index == 0:
            # Right
            slope = self.slope_of_list(data)
        if 0 < min_index < len(data) - 1:
            # Left
            slope = self.slope_of_list(data[min_index:])

        roe_finall_data = data[-1]
        roa_fuzzy, trend_fuzzy = self.fuzzy_product(roe_finall_data, slope)
        # print("ROA FUZZY:", roa_fuzzy)
        # print("TREND FUZZY:", trend_fuzzy)
        combine_rules = self.infer_rules(roa_fuzzy, trend_fuzzy)
        # print("COMBINE RULES:", combine_rules)

        final_score = self.defuzzify_sugeno(combine_rules)
        
        if get_level_label:
            level_label = self.map_fuzzy_output_centroid(final_score)
            return level_label, final_score
        return final_score

RULE_TABLE = {
    ("LOW", "DECLINING"): ("DETERIORATING", 25),
    ("LOW", "STABLE"): ("WEAK", 45),
    ("LOW", "IMPROVING"): ("NEUTRAL", 60),

    ("MEDIUM", "DECLINING"): ("WEAK", 45),
    ("MEDIUM", "STABLE"): ("NEUTRAL", 60),
    ("MEDIUM", "IMPROVING"): ("GOOD", 75),

    ("HIGH", "DECLINING"): ("NEUTRAL", 60),
    ("HIGH", "STABLE"): ("GOOD", 75),
    ("HIGH", "IMPROVING"): ("STRONG", 90),
}

# ===================== ROA =====================
# LOW    : ROA < 5%
# MEDIUM : 5% ≤ ROA < 10%
# HIGH   : ROA ≥ 10%

MAP_ROA = {
    "LOW": (
        -np.inf,
        np.nextafter(0.05, np.inf) 
    ),
    "MEDIUM": (
        0.03,
        np.nextafter(0.10, np.inf)   
    ),
    "HIGH": (
        0.08,
        np.inf
    )
}

# ===================== ROA TREND (SLOPE) =====================
# DECLINING : slope < -0.05
# STABLE    : -0.05 ≤ slope ≤ +0.05
# IMPROVING : slope > +0.05

MAP_TREND = {
    "DECLINING": (
        -np.inf,
        np.nextafter(-0.05, np.inf)
    ),
    "STABLE": (
        -0.07,
        np.nextafter(0.05, np.inf)
    ),
    "IMPROVING": (
        0.03,
        np.inf
    )
}

OUTPUT_LEVELS = {
    "DETERIORATING": 25,
    "WEAK": 45,
    "NEUTRAL": 60,
    "GOOD": 75,
    "STRONG": 90
}

fuzzy_system = ROABottomingTrendFuzzy(MAP_ROA, MAP_TREND, OUTPUT_LEVELS, RULE_TABLE)
for stock in STOCK_ID:
    try:
        level_label, final_score = fuzzy_system.ROA_Bottoming_Trend(stock, get_level_label=True)
        print(f"{stock}: Level - {level_label[0]}, Score - {final_score:.2f}")
    except Exception as e:
        print(f"{stock}: Error - {e}")

5020.T: Level - WEAK, Score - 45.00
8058.T: Level - NEUTRAL, Score - 55.67
6920.T: Level - STRONG, Score - 82.57
8001.T: Level - NEUTRAL, Score - 60.00
6952.T: Level - WEAK, Score - 45.00
4393.T: Level - STRONG, Score - 90.00
9104.T: Level - NEUTRAL, Score - 64.02
7974.T: Level - NEUTRAL, Score - 61.53
7203.T: Level - NEUTRAL, Score - 60.00
2432.T: Level - GOOD, Score - 75.00
4689.T: Level - WEAK, Score - 45.00
4063.T: Level - GOOD, Score - 71.06
5108.T: Level - NEUTRAL, Score - 59.84
9697.T: Level - GOOD, Score - 75.00
9983.T: Level - GOOD, Score - 75.00
9984.T: Level - WEAK, Score - 45.00
4568.T: Level - NEUTRAL, Score - 64.18
1812.T: Level - WEAK, Score - 49.82
7011.T: Level - WEAK, Score - 50.14
8801.T: Level - WEAK, Score - 45.00
3635.T: Level - STRONG, Score - 83.89
4816.T: Level - GOOD, Score - 75.00
8306.T: Level - WEAK, Score - 45.00
5411.T: Level - WEAK, Score - 45.00
6857.T: Level - STRONG, Score - 90.00


# 資産の換金性 - Asset Liquidity

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from itertools import product

class AssetLiquidityFuzzy:
    def __init__(self, MAP_CURRENT_RATIO, MAP_QUICK_RATIO, OUTPUT_LEVELS, RULE_TABLE):
        self.MAP_CURRENT_RATIO = MAP_CURRENT_RATIO
        self.MAP_QUICK_RATIO = MAP_QUICK_RATIO
        self.OUTPUT_LEVELS = OUTPUT_LEVELS
        self.RULE_TABLE = RULE_TABLE
    
    def in_range(self, x, a, b):
        return a < x < b
    
    def linear_fuzzy(self, x, x1, y1, x2, y2):
        if x <= min(x1, x2):
            return y1 if x1 < x2 else y2
        if x >= max(x1, x2):
            return y2 if x1 < x2 else y1
        return y1 + (y2 - y1) * (x - x1) / (x2 - x1)

    def fetch_asset_liquidity(self, stock_id):
        ticker = yf.Ticker(stock_id)
        CurentRatio = ticker.balance_sheet.loc["Current Assets"]/ticker.balance_sheet.loc["Current Liabilities"]
        QuickRatio = ticker.info.get("quickRatio")
        return CurentRatio.iloc[0], QuickRatio
    
    def Rule(self, roa, trend):
        return self.RULE_TABLE.get((roa, trend), ("UNKNOWN", 0))

    def fuzzy_product(self, current_ratio, quick_ratio):
        '''
        Fuzzification of Current Ratio and Quick Ratio
        Inputs:
            current_ratio: float, Current Ratio value
            quick_ratio: float, Quick Ratio value
        Outputs:
            CURRENT_RATIO_FUZZY: dict, fuzzy membership of Current Ratio
            QUICK_RATIO_FUZZY: dict, fuzzy membership of Quick Ratio
        '''
        
        current_ratio_raw = []
        quick_ratio_raw = []
        
        # ---------- Current Ratio fuzzification ----------
        for key, (a, b) in self.MAP_CURRENT_RATIO.items():
            if self.in_range(current_ratio, a, b):
                current_ratio_raw.append((key, a, b))
        
        CURRENT_RATIO_FUZZY = {}

        if len(current_ratio_raw) == 1:
            CURRENT_RATIO_FUZZY[current_ratio_raw[0][0]] = 1.0

        elif len(current_ratio_raw) == 2:
            (k1, a1, b1), (k2, a2, b2) = current_ratio_raw
            left = max(a1, a2)
            right = min(b1, b2)

            CURRENT_RATIO_FUZZY[k1] = self.linear_fuzzy(current_ratio, left, 1, right, 0)
            CURRENT_RATIO_FUZZY[k2] = self.linear_fuzzy(current_ratio, left, 0, right, 1)

        else:
            CURRENT_RATIO_FUZZY["UNKNOWN"] = 0.0

        # ---------- Quick Ratio fuzzification ----------
        for key, (a, b) in self.MAP_QUICK_RATIO.items():
            if self.in_range(quick_ratio, a, b):
                quick_ratio_raw.append((key, a, b))
        
        QUICK_RATIO_FUZZY = {}

        if len(quick_ratio_raw) == 1:
            QUICK_RATIO_FUZZY[quick_ratio_raw[0][0]] = 1.0

        elif len(quick_ratio_raw) == 2:
            (k1, a1, b1), (k2, a2, b2) = quick_ratio_raw
            left = max(a1, a2)
            right = min(b1, b2)

            QUICK_RATIO_FUZZY[k1] = self.linear_fuzzy(quick_ratio, left, 1, right, 0)
            QUICK_RATIO_FUZZY[k2] = self.linear_fuzzy(quick_ratio, left, 0, right, 1)
        else:
            QUICK_RATIO_FUZZY["UNKNOWN"] = 0.0
            
        return CURRENT_RATIO_FUZZY, QUICK_RATIO_FUZZY

    def infer_rules(self, current_ratio_fuzzy, quick_ratio_fuzzy):
        combine_rules = []

        for (current_ratio_label, current_ratio_w), (quick_ratio_label, quick_ratio_w) in product(
            current_ratio_fuzzy.items(), quick_ratio_fuzzy.items()
        ):
            rule_name, rule_score = self.Rule(current_ratio_label, quick_ratio_label)

            firing_strength = min(current_ratio_w, quick_ratio_w)  # AND = min

            combine_rules.append({
                "current_ratio": current_ratio_label,
                "quick_ratio": quick_ratio_label,
                "label": rule_name,
                "score": rule_score,
                "weight": firing_strength
            })

        return combine_rules

    def defuzzify_sugeno(self, combine_rules):
        numerator = 0.0
        denominator = 0.0

        for r in combine_rules:
            numerator += r["weight"] * r["score"]
            denominator += r["weight"]

        if denominator == 0:
            return 0

        return numerator / denominator

    def map_fuzzy_output_centroid(self, score):
        label = min(
            self.OUTPUT_LEVELS.items(),
            key=lambda x: abs(score - x[1])
        )
        return label

    def Asset_Liquidity_Fuzzy(self, stock_id, get_level_label=False):
        current_ratio, quick_ratio = self.fetch_asset_liquidity(stock_id)
        current_ratio_fuzzy, quick_ratio_fuzzy = self.fuzzy_product(current_ratio, quick_ratio)
        combine_rules = self.infer_rules(current_ratio_fuzzy, quick_ratio_fuzzy)
        final_score = self.defuzzify_sugeno(combine_rules)
        
        if get_level_label:
            level_label = self.map_fuzzy_output_centroid(final_score)
            return level_label, final_score
        return final_score


RULE_TABLE = {
    # --- STRONG ---
    ("HIGH", "HIGH"): ("STRONG", 90),

    # --- GOOD ---
    ("MEDIUM", "HIGH"): ("GOOD", 75),
    ("HIGH", "MEDIUM"): ("GOOD", 75),

    # --- NEUTRAL ---
    ("LOW", "HIGH"): ("NEUTRAL", 60),
    ("MEDIUM", "MEDIUM"): ("NEUTRAL", 60),
    ("HIGH", "LOW"): ("NEUTRAL", 60),

    # --- WEAK ---
    ("LOW", "MEDIUM"): ("WEAK", 45),
    ("MEDIUM", "LOW"): ("WEAK", 45),

    # --- DETERIORATING ---
    ("LOW", "LOW"): ("DETERIORATING", 25),
}

# ===================== CURRENT RATIO =====================
# LOW    : x < 1.0
# MEDIUM : 0.8 ≤ x < 1.5
# HIGH   : x ≥ 1.3

MAP_CURRENT_RATIO = {
    "LOW": (
        -np.inf,
        1.0
    ),
    "MEDIUM": (
        np.nextafter(0.8, -np.inf),
        1.5
    ),
    "HIGH": (
        np.nextafter(1.3, -np.inf),
        np.inf
    )
}

# ===================== QUICK RATIO =====================
# LOW    : x < 1.0
# MEDIUM : 0.8 ≤ x < 1.5
# HIGH   : x ≥ 1.3

MAP_QUICK_RATIO = {
    "LOW": (
        -np.inf,
        1.0
    ),
    "MEDIUM": (
        np.nextafter(0.8, -np.inf),
        1.5
    ),
    "HIGH": (
        np.nextafter(1.3, -np.inf),
        np.inf
    )
}

OUTPUT_LEVELS = {
    "DETERIORATING": 25,
    "WEAK": 45,
    "NEUTRAL": 60,
    "GOOD": 75,
    "STRONG": 90
}

fuzzy_system = AssetLiquidityFuzzy(MAP_CURRENT_RATIO, MAP_QUICK_RATIO, OUTPUT_LEVELS, RULE_TABLE)

for stock in STOCK_ID:
    try:
        level_label, final_score = fuzzy_system.Asset_Liquidity_Fuzzy(stock, get_level_label=True)
        print(f"{stock}: Level - {level_label[0]}, Score - {final_score:.2f}")
    except Exception as e:
        print(f"{stock}: Error - {e}")

5020.T: Level - NEUTRAL, Score - 62.78
8058.T: Level - NEUTRAL, Score - 67.33
6920.T: Level - NEUTRAL, Score - 66.00
8001.T: Level - WEAK, Score - 47.48
6952.T: Level - STRONG, Score - 90.00
4393.T: Level - STRONG, Score - 90.00
9104.T: Level - WEAK, Score - 45.00
7974.T: Level - STRONG, Score - 90.00
7203.T: Level - NEUTRAL, Score - 60.00
2432.T: Level - STRONG, Score - 90.00
4689.T: Level - NEUTRAL, Score - 60.00
4063.T: Level - STRONG, Score - 90.00
5108.T: Level - STRONG, Score - 86.62
9697.T: Level - STRONG, Score - 90.00
9983.T: Level - STRONG, Score - 90.00
9984.T: Level - DETERIORATING, Score - 25.00
4568.T: Level - GOOD, Score - 79.05
1812.T: Level - WEAK, Score - 47.55
7011.T: Level - WEAK, Score - 45.00
8801.T: Level - NEUTRAL, Score - 60.00
3635.T: Level - STRONG, Score - 90.00
4816.T: Level - STRONG, Score - 90.00
8306.T: Error - 'Current Assets'
5411.T: Level - NEUTRAL, Score - 60.00
6857.T: Level - STRONG, Score - 84.53


# -----------------------

In [53]:
print([-6, -1])
print(np.nextafter(-6, -np.inf))
print((np.nextafter(-1, -np.inf)))

[-6, -1]
-6.000000000000001
-1.0000000000000002


In [55]:
print([1, 6])
print(np.nextafter(1, np.inf))
print((np.nextafter(6, np.inf)))

[1, 6]
1.0000000000000002
6.000000000000001


In [ ]:
MAP_TREND = {
    "DECLINING": (
        -np.inf,
        np.nextafter(-0.05, np.inf)
    ),
    "STABLE": (
        -0.07,
        np.nextafter(0.05, np.inf)
    ),
    "IMPROVING": (
        0.03,
        np.inf
    )
}

LOW_AREA ={(-np.inf, 1), (-0.07, 1), (-0.05, 0), (-np.inf, 0)}

In [15]:
import yfinance as yf

stock_id = "8306.T"
ticker = yf.Ticker(stock_id)

info = ticker.info

print("Sector:", info.get("sector"))
print("Industry:", info.get("industry"))

Sector: Financial Services
Industry: Banks - Diversified


In [13]:
stock_id = "8306.T"

ticker = yf.Ticker(stock_id)

In [16]:
def compute_current_assets(bs):
    for key in ["Total Current Assets", "Current Assets"]:
        if key in bs.index:
            return bs.loc[key].iloc[0]
    components = [
        "Cash And Cash Equivalents",
        "Short Term Investments",
        "Net Receivables",
        "Inventory",
        "Prepaid Expenses",
        "Other Current Assets"
    ]

    total = 0
    found = False

    for key in components:
        if key in bs.index:
            total += bs.loc[key].iloc[0]
            found = True

    return total if found else None

current_assets = compute_current_assets(ticker.balance_sheet)
print("Current Assets:", current_assets)

Current Assets: 109095437000000.0


In [ ]:
def compute_current_assets(bs):
    for key in ["Total Current Assets", "Current Assets"]:
        if key in bs.index:
            return bs.loc[key].iloc[0]
    components = [
        "Cash And Cash Equivalents",
        "Short Term Investments",
        "Net Receivables",
        "Inventory",
        "Prepaid Expenses",
        "Other Current Assets"
    ]

    total = 0
    found = False

    for key in components:
        if key in bs.index:
            total += bs.loc[key].iloc[0]
            found = True

    return total if found else None

def compute_current_liabilities(bs):
    for key in ["Total Current Liabilities", "Current Liabilities"]:
        if key in bs.index:
            return bs.loc[key].iloc[0]

    components = [
        "Accounts Payable",
        "Short Term Debt",
        "Current Debt",
        "Accrued Liabilities",
        "Other Current Liabilities"
    ]

    total = 0
    found = False

    for key in components:
        if key in bs.index:
            total += bs.loc[key].iloc[0]
            found = True

    return total if found else None

stock_id = "8306.T"
ticker = yf.Ticker(stock_id)
bs = ticker.balance_sheet
current_assets = compute_current_assets(bs)
current_liabilities = compute_current_liabilities(bs)
print("Current Assets:", current_assets)
print("Current Liabilities:", current_liabilities)
print("Current Ratio:", current_assets / current_liabilities)

Current Assets: 109095437000000.0
Current Liabilities: nan
Current Ratio: nan


In [ ]:
stock_id = "1812.T"
ticker = yf.Ticker(stock_id)
current_assets = ticker.balance_sheet.loc["Current Assets"]
current_liabilities = ticker.balance_sheet.loc["Current Liabilities"]
print("Current Assets:", current_assets)
print("Current Liabilities:", current_liabilities)
CurentRatio = current_assets / current_liabilities  
QuickRatio = ticker.info.get("quickRatio")
print()
print("--- Asset Liquidity Calculation ---")
print("Current Ratio:", CurentRatio.iloc[0])
print("Quick Ratio:", QuickRatio)

Current Assets: 2025-03-31    2.137129e+12
2024-03-31    1.917988e+12
2023-03-31    1.751584e+12
2022-03-31    1.390711e+12
Name: Current Assets, dtype: float64
Current Liabilities: 2025-03-31    1.696974e+12
2024-03-31    1.506000e+12
2023-03-31    1.319768e+12
2022-03-31    1.107668e+12
Name: Current Liabilities, dtype: float64

--- Asset Liquidity Calculation ---
Current Ratio: 1.2593763958669961
Quick Ratio: 0.834


In [10]:
stock_id = "8306.T"
ticker = yf.Ticker(stock_id)
bs = ticker.balance_sheet
quick_ratio = ticker.info.get("quickRatio")
print("Quick Ratio:", quick_ratio)

Quick Ratio: None
